# <b>Machine</b> <b>L</b>earning <b>Appl</b>ied to <b>Vio</b>lence <b>De</b>tection in <b>Sur</b>veillance <b>Videos</b> (MachineLApplVioDeSurVideos)

## Development environment setup

In [ ]:
%%sx
#@title Colab development environment settings
if [ -n "${COLAB_RELEASE_TAG}" ]; then
    git clone -b main https://github.com/MigeoDaSelva/MachineLApplVioDeSurVideos.git;
    python3 -m pip install --upgrade pip;
    python3 -m pip install -r /content/MachineLApplVioDeSurVideos/requirements.txt;
    kill -9 pid "${PPID}";
fi

In [ ]:
#@title Import libraries
from pprint import pprint
import importlib
import sys
import os

In [ ]:
#@title System path manager
%reload_ext autoreload
%autoreload 2

project_name = "MachineLApplVioDeSurVideos"

working_directory = os.popen("echo $PWD").read().rstrip()

project_root_path = working_directory \
    if project_name in working_directory \
        else os.popen(
            f"readlink -f $(find -name {project_name} \
                -not -path '*/drive/*')"
        ).read().rstrip()

if str(project_root_path) not in sys.path:
    sys.path.insert(0, str(project_root_path))

pprint(f"Environment paths: {sys.path}")

In [ ]:
#@title Import project stuff
from src.application.evaluation_application import EvaluationApplication
from src.application.execution_application import ExecutionApplication
from src.application.startup_application import StartupApplication
from configs import settings

In [ ]:
#@title Colab resource settings
if settings.COLAB_ENV: 
    from google.colab import output
    from google.colab import drive
    output.enable_custom_widget_manager()
    drive.mount("/content/drive", force_remount=True)

In [ ]:
#@title Define simulation scope
importlib.reload(settings)
simulation_scope_settings = StartupApplication.get_scope_manager()

In [ ]:
#@title Select data settings
importlib.reload(settings)
data_settings = StartupApplication.get_data_manager()

In [ ]:
#@title Select approach settings
importlib.reload(settings)
approach_settings = StartupApplication.get_approach_manager()

In [ ]:
#@title Show selected values

print(
    "\nSimulation scope sttings:\n",
    f"Iterations: {simulation_scope_settings.iterations}",
    f"folds: {simulation_scope_settings.folds}",
    sep="\n"
)

print(
    "\nSelected data attributes:\n",
    f"Number of channels: {data_settings.n_channels}",
    f"Number of frames: {data_settings.n_frames}",
    f"Image height: {data_settings.image_height}",
    f"Image width: {data_settings.image_width}",
    f"Batch size: {data_settings.batch_size}",
    sep = "\n"
)

print(
    f"\nSelected {approach_settings.approach.__name__} attributes:\n",
    f"Callbacks: {[callback.__class__.__name__ for callback in approach_settings.callbacks]}",
    f"Number of output initial filters: {approach_settings.filters}",
    f"Pooling Strides: {approach_settings.pooling_strides}",
    f"Activation function: {approach_settings.activation}",
    f"Padding approach: {approach_settings.padding}",
    f"Kernel size: {approach_settings.kernel_size}",
    f"Load best model: {approach_settings.continue_training}",
    f"Number of epochs: {approach_settings.epochs}",
    f"Learning rate: {approach_settings.learning_rate}",
    f"Unfreezing: {approach_settings.unfreezing}",
    f"Optimizer: {approach_settings.optimizer.__name__}",
    f"Dropout: {approach_settings.dropout}",
    sep = "\n"
)

In [ ]:
#@title build pipeline
importlib.reload(settings)
pipeline = ExecutionApplication.get_pipeline(
    simulation_scope_settings=simulation_scope_settings,
    data_settings=data_settings,
    approach_settings=approach_settings
)

In [ ]:
#@title Pipeline runimg
importlib.reload(settings)
pipeline.run()

In [ ]:
#@title Load latest best model
importlib.reload(settings)
ExecutionApplication.load_best_model(pipeline.approach)

In [ ]:
#@title Evaluate
importlib.reload(settings)
evaluation_application = EvaluationApplication(
    approach=pipeline.approach,
    data_settings=data_settings,
    simulation_scope_settings=simulation_scope_settings,
)

evaluation_application.evaluate()
evaluation_application.calculate_classification_metrics()
evaluation_application.save_confusion_matrix()

In [ ]:
#@title Post-training
importlib.reload(settings)
evaluation_application.calculate_average_classification_metrics()
evaluation_application.build_confusion_matrix()